In [5]:
from multiprocessing import Pool 
import csv 
from datetime import datetime 
import os 
import requests 
import pandas as pd 
import selenium 
from selenium.webdriver import Chrome 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException, WebDriverException 
from user_agent import generate_user_agent 
from time import sleep 
#from text_processing import change_date 
#from text_processing import lemmatizator 
import numpy as np 
from os import getcwd

In [42]:
import urllib 
from urllib.parse import unquote

In [29]:
browser = Chrome(executable_path = '/home/twsea/Documents/python-mail/PrPythonAtom/homeworks/project/chromedriver')
browser.get('https://www.kinopoisk.ru/')
search_form = browser.find_element_by_class_name('header-fresh-search-partial-component__field')
search_form.send_keys('Елки')
search_form.send_keys(Keys.ENTER)
search_form = browser.find_element_by_xpath('//*[@id="block_left_pad"]/div/div[2]/div/div[2]/p/a') #клик по первому фильму
search_form.click()
browser.close()


In [47]:
string = '%E5%EB%EA%E8'

In [58]:
import urllib.parse

s = 'елки'
urllib.parse.quote(s.encode('utf-8'))

'%D0%B5%D0%BB%D0%BA%D0%B8'

In [59]:
!pip install requests bs4 user_agent selenium

In [202]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [227]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to /home/twsea/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/twsea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [203]:
#рассматриваем две ситуации:
#либо такого фильма не существует, тогда вылетает класс document.getElementsByClassName('textgreybig')
#либо фильм существует

In [205]:
name = 'елки'
url = 'https://www.kinopoisk.ru/index.php?kp_query=' + name +'&what='
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
if (soup.findAll("p", attrs={"class": ["name"]})): #если фильм существует
    href = soup.findAll("p", attrs={"class": ["name"]})[0].find('a')['href'] #ссылка на искомый фильм
    title = soup.findAll("p", attrs={"class": ["name"]})[0].find('a').text #название фильма
    href = href[:len(href)-5] #обрезаю ссылку
    #https://www.kinopoisk.ru/film/yolki-2010-493768/ord/rating/#list
    #print(href)
    new_url = 'https://www.kinopoisk.ru' + href + 'ord/rating/#list' #ссылка на отсортированные комменты (кнопка "Все")
    #print(new_url)
    #print(title)
    r = requests.get(new_url)
    soup = BeautifulSoup(r.text, "html.parser")
    comments = []
    for i in range(len(soup.findAll("div", attrs={"itemprop": ["reviews"]}))):
        comments.append(soup.findAll("div", attrs={"itemprop": ["reviews"]})[i].findAll("span", attrs={"class": ["_reachbanner_"]})[0].text)
        comments[i] = comments[i].replace('\n', ' ').replace('\r', '').replace('\t', '').replace(u'\xa0', u' ').replace('\x97', '—').replace('\x85', '...').replace('\x88', '^')
else:
    print('Введеное название фильма не найдено.')

In [228]:
def parsing_reviews (film_name): #возвращает массив типа [название, отзывы, оценка]
    url = 'https://www.kinopoisk.ru/index.php?kp_query=' + film_name +'&what='
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    if (soup.findAll("p", attrs={"class": ["name"]})): #если фильм существует
        href = soup.findAll("p", attrs={"class": ["name"]})[0].find('a')['href'] #ссылка на искомый фильм
        title = soup.findAll("p", attrs={"class": ["name"]})[0].find('a').text #название фильма
        href = href[:len(href)-5] #обрезаю ссылку
        new_url = 'https://www.kinopoisk.ru' + href + 'ord/rating/#list' #ссылка на отсортированные комменты (кнопка "Все")
        r = requests.get(new_url)
        soup = BeautifulSoup(r.text, "html.parser")
        comments = []
        #беру только 2 отзыва о фильме
        length = 2 if len(soup.findAll("div", attrs={"itemprop": ["reviews"]})) else len(soup.findAll("div", attrs={"itemprop": ["reviews"]}))
        for i in range(length):
            comments.append(soup.findAll("div", attrs={"itemprop": ["reviews"]})[i].findAll("span", attrs={"class": ["_reachbanner_"]})[0].text)
            comments[i] = comments[i].replace('\n', ' ').replace('\r', '').replace('\t', '').replace(u'\xa0', u' ').replace('\x97', '—').replace('\x85', '...').replace('\x88', '^')
        res = ''
        for i in comments: #запихиваю 2 отзыва в 1 строку
            res = res + i
        assessment = float(soup.findAll("span", attrs={"class": ["rating_ball"]})[0].text) 
        assessment = 0 if assessment < 6 else 1
        return [film_name, res, assessment]
    else:
        print('Введеное название фильма не найдено.') 

In [229]:
parsing_reviews('Елки')

['Елки',
 'Абсолютно ничего не ждав от данной картины, я решила не тратить свое время и пропустить просмотр «очередной русской комедии». Но вот четверг, утро, мне нужно скоротать 2 часа свободного времени. Альтернатив «Елкам» не нашлось, пришлось идти. Что же я увидела?  Во-первых, к своему удивлению фильм затянул, затянул в свою праздничную атмосферу. Ведь никогда мы не бываем столь близки к волшебству, как в Новый Год. Кстати, у большинства россиян Новый год является любимым праздником, в том числе и у меня, поэтому вся предновогодняя суета отражается в моем сердце яркими огоньками! И было чертовски приятно смотреть на происходящее на экране и осознавать, что любимый праздник уже через две недели!  Во-вторых, я с радостью обнаружила, что фильм не пародия, не римейк, не «оригинальный фильм, просто слизанный с ранее вышедшей зарубежной картины». В общем, в плане сценария провисаний не заметила. Может быть они и были, но, вы знаете, как-то не хочется их искать.  В-третьих, понравилось, 

In [ ]:
#нужны рейтинги измененные
#нужно напарсить данные и разбить их на обучение и трейн.
#как плохие фильмы брать, так и хорошие

In [207]:
film_name = "елки"
url = 'https://www.kinopoisk.ru/index.php?kp_query=' + film_name +'&what='
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
if (soup.findAll("p", attrs={"class": ["name"]})): #если фильм существует
    href = soup.findAll("p", attrs={"class": ["name"]})[0].find('a')['href'] #ссылка на искомый фильм
    title = soup.findAll("p", attrs={"class": ["name"]})[0].find('a').text #название фильма
    href = href[:len(href)-5] #обрезаю ссылку
    new_url = 'https://www.kinopoisk.ru' + href + 'ord/rating/#list' #ссылка на отсортированные комменты (кнопка "Все")
    r = requests.get(new_url)
    soup = BeautifulSoup(r.text, "html.parser")
else:
    print('Введеное название фильма не найдено.') 

In [215]:
float(soup.findAll("span", attrs={"class": ["rating_ball"]})[0].text)

6.877

In [ ]:
url = 'https://www.kinopoisk.ru/index.php?kp_query=' + film_name +'&what='
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")